In [1]:
#6GSRTC
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from decimal import Decimal
import pandas as pd
import time


driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20) 

bus_route_link = []
bus_route_name = []
bus_route_final = []
#bus_route = []
bus_names = [] 
bus_type = [] 
departing_time = []
duration = [] 
Reaching_time = [] 
star_rating = [] 
price = [] 
seat_availablity = []

def jump_pages():
    for page_number in range(1,6):
        find_bus_routes = driver.find_elements(By.CLASS_NAME,'route')
        if page_number < 6:
            for route in find_bus_routes:
                route_name = route.get_attribute("title")
                route_link = route.get_attribute("href")
                bus_route_name.append(route_name)
                bus_route_link.append(route_link)
            try:
                pagination_container = wait.until(EC.presence_of_element_located((By.XPATH, '//div[contains(@class, "DC_117_paginationTable")]')))
                next_page_button=pagination_container.find_element(By.XPATH, f'//div[contains(@class, "DC_117_pageTabs") and text()="{page_number + 1}"]')
                action = ActionChains(driver)
                action.move_to_element(next_page_button).perform()
                time.sleep(2)
                next_page_button.click()
            except:
                print('failure')

def scroll(): # function to scroll page from top to bottom
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")# Scroll down to the bottom in order to load all the buses
        time.sleep(2)# Wait for page to load
        new_height = driver.execute_script("return document.body.scrollHeight")# Calculate new scroll height and compare with last scroll height
        if new_height == last_height:# If it is the same height then it is at the end of the page
            break
        last_height = new_height

def click_button(): # function to click button for view buses for government buses
    view_buses = driver.find_elements(By.CSS_SELECTOR, '.p-left-10.icon.icon-down')
    if view_buses:
        button_count = len(view_buses)
        for i in range(button_count-1 , -1, -1):
            view_buses[i].click()

def get_bus_details(): # function to get all bus details
    rate = driver.find_elements(By.CSS_SELECTOR,".fare.d-block")#
    seat_available = driver.find_elements(By.CSS_SELECTOR,".column-eight.w-15.fl")
    rating = driver.find_elements(By.CSS_SELECTOR,".column-six.p-right-10.w-10.fl")
    names = driver.find_elements(By.CSS_SELECTOR, ".travels.lh-24.f-bold.d-color")
    b_type = driver.find_elements(By.CSS_SELECTOR,".bus-type.f-12.m-top-16.l-color.evBus")
    start_time = driver.find_elements(By.CSS_SELECTOR,".dp-time.f-19.d-color.f-bold")
    total_time = driver.find_elements(By.CSS_SELECTOR,".dur.l-color.lh-24")
    end_time =  driver.find_elements(By.CSS_SELECTOR,".bp-time.f-19.d-color.disp-Inline")
    
    time.sleep(5)
    print(len(names),len(b_type),len(start_time),len(total_time),len(end_time),len(rating),len(rate),len(seat_available))

    for i in range(0,len(seat_available)):#(len(rating)):
        bus_names.append(names[i].text)
        bus_type.append(b_type[i].text)
        departing_time.append(start_time[i].text)
        duration.append(total_time[i].text)
        Reaching_time.append(end_time[i].text)
        if (rating[i].text) == "New":
            star_rating.append(0.0)
        else:
            star_rating.append((rating[i].text)[0:3])
        price.append((rate[i].text))
        seat_availablity.append((seat_available[i].text)[0:2])
    
    print(len(bus_names),len(bus_type),len(departing_time),len(duration),len(Reaching_time),len(star_rating),len(price),len(seat_availablity))
    print((bus_names),(bus_type),(departing_time),(duration),(Reaching_time),(star_rating),(price),(seat_availablity))


driver.get("https://www.redbus.in/online-booking/gsrtc")
driver.maximize_window()
time.sleep(5)

jump_pages()

print("Total routes",len(bus_route_name))

for i in range (0,len(bus_route_link)):# used 2 just to check if every thing is working as expected
    driver.get(bus_route_link[i])
    body = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.clearfix.bus-item')))
    click_button()
    scroll()
    time.sleep(5)
    get_bus_details()
    temp_name = []
    temp_link = []
    bus_route = []
    state_service = []
    print(len(bus_names))
    for j in range(len(bus_names)):
        temp_name.append(bus_route_name[i])
        temp_link.append(bus_route_link[i])
        state_service.append("GSRTC")
    print(len(temp_name),len(temp_link))
    bus_route = list(zip(state_service,temp_name,temp_link,bus_names,bus_type,departing_time,duration,Reaching_time,star_rating,price,seat_availablity))
    bus_route_final.extend(bus_route)
    print(i+1,bus_route_name[i],"completed")
    temp_name.clear()
    temp_link.clear()
    state_service.clear()
    bus_route.clear()
    bus_names.clear()
    bus_type.clear()
    departing_time.clear()
    duration.clear()
    Reaching_time.clear()
    star_rating.clear()
    price.clear()
    seat_availablity.clear()
    
df_bus_details_1 = pd.DataFrame(bus_route_final, columns=["Service Name","Route_name", "Route_link", "Bus_names","Bus_type","Departing_time","Duration","Reaching_time","Star_rating","Price","Seat_availablity"])
df_bus_details_1['Seat_availablity']=df_bus_details_1['Seat_availablity'].astype(int)
df_bus_details_1['Price'] = df_bus_details_1['Price'].str.replace('INR ', '', regex=False)
df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].str.replace(' ', '', regex=False)
df_bus_details_1['Price'] = df_bus_details_1['Price'].astype(float)
df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].fillna(0.0)

failure
Total routes 47
1 1 1 1 1 1 1 1
1 1 1 1 1 1 1 1
['Royal Karnavati Travels'] ['A/C Seater / Sleeper (2+1)'] ['15:15'] ['19h 15m'] ['10:30'] ['3.4'] ['INR 1199'] ['41']
1
1 1
0 Ahmedabad to Ganganagar (Sri Ganganagar) completed
8 8 8 8 8 8 8 8
8 8 8 8 8 8 8 8
['Mahaveer Travel Company', 'Rytham Tours And Travels', 'Royal Karnavati (TNB Travels)', 'Milan Travels', 'Parshwanath Travel Pvt. Ltd', 'Royal Karnavati Travels', 'Salasar vaishnav travels', 'RMB Travel Agency'] ['NON AC Seater / Sleeper 2+1', 'NON A/C Sleeper (2+1)', 'A/C Seater / Sleeper (2+1)', 'A/C Seater / Sleeper (2+1)', 'Bharat Benz A/C Sleeper (2+1)', 'A/C Seater / Sleeper (2+1)', 'A/C Seater / Sleeper (2+1)', 'NON A/C Seater/ Sleeper (2+1)'] ['23:00', '23:35', '23:30', '23:43', '23:55', '21:00', '23:56', '22:15'] ['08h 00m', '07h 00m', '08h 15m', '07h 17m', '07h 35m', '07h 35m', '07h 50m', '07h 45m'] ['07:00', '06:35', '07:45', '07:00', '07:30', '04:35', '07:46', '06:00'] ['1.7', '1.8', '4.7', '3.9', '3.8', '4.7', 

In [13]:
pd.set_option('display.max_rows',None)
df_bus_details_1

,Service Name,Route_name,Route_link,Bus_names,Bus_type,Departing_time,Duration,Reaching_time,Star_rating,Price,Seat_availablity
0,GSRTC,Ahmedabad to Ganganagar (Sri Ganganagar),https://www.redbus.in/bus-tickets/ahmedabad-to...,Royal Karnavati Travels,A/C Seater / Sleeper (2+1),15:15,19h 15m,10:30,3.4,1199.00,41
1,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Mahaveer Travel Company,NON AC Seater / Sleeper 2+1,23:00,08h 00m,07:00,1.7,500.00,21
2,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Rytham Tours And Travels,NON A/C Sleeper (2+1),23:35,07h 00m,06:35,1.8,700.00,36
3,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Royal Karnavati (TNB Travels),A/C Seater / Sleeper (2+1),23:30,08h 15m,07:45,4.7,699.00,38
4,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Milan Travels,A/C Seater / Sleeper (2+1),23:43,07h 17m,07:00,3.9,760.00,17
5,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Parshwanath Travel Pvt. Ltd,Bharat Benz A/C Sleeper (2+1),23:55,07h 35m,07:30,3.8,904.00,15
6,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Royal Karnavati Travels,A/C Seater / Sleeper (2+1),21:00,07h 35m,04:35,4.7,599.00,29
7,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,Salasar vaishnav travels,A/C Seater / Sleeper (2+1),23:56,07h 50m,07:46,2.2,800.00,46
8,GSRTC,Sirohi (Rajasthan) to Bikaner,https://www.redbus.in/bus-tickets/sirohi-to-bi...,RMB Travel Agency,NON A/C Seater/ Sleeper (2+1),22:15,07h 45m,06:00,2.2,500.00,30
9,GSRTC,Ahmedabad to Bikaner,https://www.redbus.in/bus-tickets/ahmedabad-to...,RSRTC - 191942,Express Non AC Seater 2+3,17:30,17h 30m,11:00,3.6,733.00,42


In [15]:
#df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].astype(float)
df_bus_details_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Service Name      1462 non-null   object 
 1   Route_name        1462 non-null   object 
 2   Route_link        1462 non-null   object 
 3   Bus_names         1462 non-null   object 
 4   Bus_type          1462 non-null   object 
 5   Departing_time    1462 non-null   object 
 6   Duration          1462 non-null   object 
 7   Reaching_time     1462 non-null   object 
 8   Star_rating       1462 non-null   object 
 9   Price             1462 non-null   float64
 10  Seat_availablity  1462 non-null   int32  
dtypes: float64(1), int32(1), object(9)
memory usage: 120.1+ KB


In [17]:
df_bus_details_1['Star_rating'] = df_bus_details_1['Star_rating'].astype(float)

ValueError: could not convert string to float: ''

In [21]:
df_bus_details_1.to_csv("GSRTC.csv",index = False)